In [13]:
!pip install --quiet transformers accelerate
!pip install --quiet torchaudio
!pip install --quiet ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00


In [3]:
import torch
import torchaudio
import numpy as np
from transformers import AutoProcessor, SeamlessM4Tv2ForSpeechToSpeech
from IPython.display import Audio, display
import time

In [4]:
# --- Setup for SeamlessM4T v2 Model ---
# 1. Load the pre-trained model and processor.
model_name = "facebook/seamless-m4t-v2-large"
processor = AutoProcessor.from_pretrained(model_name)
model = SeamlessM4Tv2ForSpeechToSpeech.from_pretrained(model_name)

# Set the device to GPU if available.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Instantiating a decoder SeamlessM4Tv2Attention without passing `layer_idx` is not recommended and will lead to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` when creating this class.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

SeamlessM4Tv2ForSpeechToSpeech(
  (shared): Embedding(256102, 1024, padding_idx=0)
  (speech_encoder): SeamlessM4Tv2SpeechEncoder(
    (feature_projection): SeamlessM4Tv2ConformerFeatureProjection(
      (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=160, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SeamlessM4Tv2ConformerEncoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-23): 24 x SeamlessM4Tv2ConformerEncoderLayer(
          (ffn1_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (ffn1): SeamlessM4Tv2ConformerFeedForward(
            (intermediate_dropout): Dropout(p=0.0, inplace=False)
            (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
            (intermediate_act_fn): SiLU()
            (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
      

In [19]:
def convert_audio(audio_file, src_lang, tgt_lang, target_sampling_rate = 16000):
  print(f"Translating from {src_lang} to {tgt_lang}...")
  # The audio is a webm file; torchaudio can read this
  audio_tensor, sampling_rate = torchaudio.load(audio_file)

  # The model expects mono audio, so we'll convert it if needed.
  if audio_tensor.shape[0] > 1:
    audio_tensor = torch.mean(audio_tensor, dim=0, keepdim=True)

  # The model expects a 16 kHz sampling rate. We must resample if needed.
  if sampling_rate != target_sampling_rate:
      transform = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
      audio_tensor = transform(audio_tensor)
      sampling_rate = target_sampling_rate

  # --- Pre-processing and Translation ---
  print("Translating audio...")
  audio_inputs = processor(
      audios=audio_tensor,
      sampling_rate=sampling_rate,
      src_lang=src_lang,
      return_tensors="pt"
  ).to(device)

  print("Still working...")

  output_audio_array = model.generate(**audio_inputs, tgt_lang=tgt_lang)[0].cpu().numpy().squeeze()
  print("Translation finished.")
  return output_audio_array

In [7]:
def display_audio(audio_array, sampling_rate, language):
  # --- Display Results ---
  # output_sampling_rate = model.config.sampling_rate
  print(f"Video in ${language}...")
  # display(Audio(output_audio_array, rate=output_sampling_rate))
  display(Audio(audio_array, rate=sampling_rate))

In [21]:
#This block will create interactive widgets to test the feature as a app

import ipywidgets as widgets
from IPython.display import clear_output, Javascript
from base64 import b64decode
import io
from google.colab import output

LANGUAGES = {
    "English": "eng",
    "Spanish": "spa",
    "French": "fra",
    "German": "deu",
    "Hindi": "hin",
    "Bengali": "ben",
    "Mandarin Chinese": "cmn",
    "Japanese": "jpn",
}

# --- Create Interactive Widgets ---

# Dropdowns for language selection
source_lang_dropdown = widgets.Dropdown(
    options=LANGUAGES.keys(),
    value="English",
    description="Source Language:",
    disabled=False,
)

target_lang_dropdown = widgets.Dropdown(
    options=LANGUAGES.keys(),
    value="Spanish",
    description="Target Language:",
    disabled=False,
)

# Button to trigger the recording and translation process
record_button = widgets.Button(
    description="Record & Translate",
    button_style="success",
    tooltip="Click to start recording audio",
    icon="microphone",
)

# Progress bar to show conversion status
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Status:',
    bar_style='info',
    style={'bar_color': 'green'},
    layout=widgets.Layout(width='auto')
)


# Output area for displaying results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([
    widgets.HBox([source_lang_dropdown, target_lang_dropdown]),
    record_button,
    # progress_bar,
])

display(input_widgets, output_area)


# --- JavaScript for Audio Recording ---
JS_CODE = """
async function recordAudio() {
  const div = document.createElement('div');
  const button = document.createElement('button');
  const stopButton = document.createElement('button');

  button.textContent = 'Record';
  stopButton.textContent = 'Stop';
  stopButton.disabled = true;

  const stream = await navigator.mediaDevices.getUserMedia({audio: true});
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);

  recorder.start();
  button.disabled = true;
  stopButton.disabled = false;

  stopButton.onclick = () => {
    recorder.stop();
    button.disabled = false;
    stopButton.disabled = true;
    div.remove();
  };

  div.appendChild(button);
  div.appendChild(stopButton);
  document.body.appendChild(div);

  await new Promise(resolve => recorder.onstop = resolve);

  const blob = new Blob(chunks, {'type' : 'audio/webm;codecs=opus'});
  const url = URL.createObjectURL(blob);
  const reader = new FileReader();
  reader.readAsDataURL(blob);
  await new Promise(resolve => reader.onload = () => resolve());

  const data_url = reader.result;
  const audio_data = data_url.substr(data_url.indexOf(',') + 1);
  return audio_data
}
"""

def on_button_clicked(b):
    with output_area:
        clear_output()

        print("Recording audio...")
        # Execute the JavaScript code and capture the audio data
        recorded_b64 = output.eval_js(JS_CODE + 'recordAudio()')


        # Check if recording was successful
        if not recorded_b64:
            print("Recording failed or was cancelled.")
            return

        clear_output()

        # Decode the audio and load it
        audio_bytes = b64decode(recorded_b64)
        audio_file = io.BytesIO(audio_bytes)

        # Convert language code to language name for display
        src_lang_name = source_lang_dropdown.value
        tgt_lang_name = target_lang_dropdown.value

        display_audio(audio_bytes, 16000, src_lang_name) # Pass language name for display
        output_audio_array = convert_audio(audio_file, LANGUAGES[src_lang_name], LANGUAGES[tgt_lang_name]) # Pass language codes for model
        display_audio(output_audio_array, model.config.sampling_rate, tgt_lang_name) # Pass language name for display


# 5. Link the button to the function
record_button.on_click(on_button_clicked)

Output()